***

# Extraindo dados tabulares de um PDF

***
**Autor:** Jorge Luiz Figueira

**E-mail:** jorgeluizfigueira@gmail.com

**Linkedin:** https://www.linkedin.com/in/jorgeluizfigueira/

### Sobre este projeto

Cerca de 90% dos dados produzidos são **Não-Estruturados**. Isto é, não estão em um formato de fácil compreensão e manipulação como arquivos CSV, excel e banco de dados.  Documentos em PDF são exemplos de dados não estruturados. É importante que empresas encontrem maneiras de análisar esses dados e extrair *insights* valiosos

Este Python Notebook demonstra dois diferentes métodos para extração de dados tabulares de arquivos em PDF. 

A motivação deste projeto surgiu ao lidar com problemas de uma empresa que possui um sistema legado, com quase 20 anos em produção.

Abaixo, consta uma pré-visualização do arquivo PDF que iremos extrair os dados.

**Uma observação importante:** os dados sensíveis foram anonimizados.

![alt text](exemplo_pdf1.png "Exemplo")

## Importação das bibliotecas

In [1]:
from glob import glob

import pandas as pd
import re
import json

#Extração automática
import tabula

#Extração Manual
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

## Método Extração Automática

In [2]:
def automatic_extraction(file):

    #Extrai as tabelas presentes no pdf
    df = tabula.read_pdf(file, pages='all', multiple_tables=True,silent=True)
    
    #Verifica se a tabela ultrapassa uma página e as combinam caso ocorra.
    df_array = []
    for page in df:
        rows = len(page)
        page = page[1:rows]
        df_array.append(page)
        
    #Une as tabelas existentes
    df = pd.concat(df_array)
    
    #Preenche os valores "NAN"
    df = df.fillna(" ")
    
    #Reconfigura o index da tabela unificada
    df = df.reset_index()
    
    #Remove as colunas index e Linha
    df = df.drop(labels=['index','LINHA'],axis=1)
    
    #Corrige os nomes das colunas pelo segundo valor
    df = df.rename(columns={"VALOR": "VALOR TOTAL","VALOR.1": "VALOR ICMS ST","OPER.": "OPER. TRIANG.", "DATA": "DATA PROMETIDA"})
    
    #Ajusta os índices, para que comece a partir de 1
    df.index += 1

    #Especifica a ordem das colunas da tabela
    columnsTitles = ['REFERENCIA','QUANT.','DATA PLAN.','DATA PROMETIDA','VALOR UNIT.',
                   'VALOR TOTAL','VALOR ICMS ST','%ICMS ST','%IPI',
                  'OPER. TRIANG.','UM','CST']
    df = df.reindex(columns=columnsTitles)

    return df

### Método Extração Manual

E quando a extração automática (feito utilizando a lib **Tabula-py**) falha, como podemos lidar?

Decidi explorar outra lib, chamada **PdfMiner**, para extrair somente a parte textual (*strings*). Uma hipótese é que a partir desse texto extraído fosse possível identificar elementos que indicassem o ínicio e o fim de cada coluna da tabela presente no Arquivo PDF.

A hipótese estava correta. O ínicio e o fim de cada coluna seguia um determinado padrão. Decidi inserir marcadores nesses elementos ("<BREAK_{}>") para posteriormente selecioná-los com maior facilidade.

Esses padrões foram salvos em um arquivo TXT com o nome *"settings"*. No qual são carregados e utilizados como dicionários.

Cada chave desse dicionário representa uma coluna da tabela. E seu valor indica o par de marcadores *"BREAK"* em que os itens dessa respectiva coluna estão presentes.

In [3]:
#Utilizada quando o método automático falha
#Funcão que extrai os elementos textuais de um PDF
#Implementada a partir da definição dada  documentação oficial
#https://pdfminersix.readthedocs.io/en/latest/tutorial/composable.html

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [4]:
def manual_extraction(file):

    text = convert_pdf_to_txt(file)

    #ajuda na manipulação de strings
    text = text.replace("DATA PLAN.\nFATURAM.","DATA_PLAN._FATURAM.")
    text = text.replace("DATA \nPROMETIDA","DATA_PROMETIDA")
    text = text.replace("TOTAL MERCADORIA","TOTAL_MERCADORIA")
    text = text.replace("TOTAL IPI","TOTAL_IPI")
    text = text.replace("TOTAL ICMS ST","TOTAL_ICMS_ST")
    text = text.replace("VALOR\nICMS ST","VALOR_ICMS_ST")
    text = text.replace("%ICMS ST","%ICMS_ST")
    text = text.replace("OPER. TRIANG.","OPER._TRIANG.")
    text = text.replace("TOTAL PEDIDO","TOTAL_PEDIDO")
    text = text.replace("PEDIDO CLIENTE LINHA","PEDIDO_CLIENTE_LINHA")
    text = text.replace("VALOR UNIT.","VALOR_UNIT.")
    text = text.replace("VALOR\nTOTAL","VALOR_TOTAL")
    text = text.replace("TOTAIS POR DATA PLANEJADA DE FATURAMENTO","FIM_TABELA")

    #quebra as strings em lista
    text = text.split("\n")

    #recorta somente a parte da tabela e despreza o header
    text = text[text.index("REFERENCIA"):len(text)]

    for w in range(len(text)):
        if text[w] == "":
            text[w] = "<BREAK>"

    text = [w.strip() for w in text]

    for w in range(len(text)):
        if text[w] == "":
            text[w] = "<BLANKFILL>"

    i_break = 0
    for w in range(len(text)):
        if text[w] == "<BREAK>":
            text[w] = "<BREAK_{}>".format(i_break)
            i_break+=1

    settings = glob('settings*.txt')
    
    df = extract_fields(text,settings,0)
    
    

    return df

In [5]:
def extract_fields(text,settings,i):
    
    #Seleciona os elementos que estão entre os marcadors Break
    def items_between(value):
        items = text[text.index(f'<BREAK_{value}>')+1:text.index(f'<BREAK_{value+1}>')]
        return items
    
    #Carrega o dicionário de padrões
    with open(settings[i]) as json_file:
        data = json.load(json_file)
    n_settings = len(settings)-1
    
    global result 
    
    
    try:
        columns = {}
        
        for columm in list(data):
            columns[columm] = items_between(data[columm])
        
        #cria um dataframe
        df = pd.DataFrame({'REFERENCIA':columns['refs'],'QUANT.':columns['quant'],'DATA PLAN.':columns['date_p'],
                           'DATA PROMETIDA':columns['date_pr'],'VALOR UNIT.':columns['valor_u'],
                           'VALOR TOTAL':columns['valor_t'],'VALOR ICMS ST':columns['valor_icms'],
                           '%ICMS ST':columns['icms_st'],'%IPI':columns['ipi'],
                           'OPER. TRIANG.':columns['oper_t'],'UM':columns['um'],'CST':columns['cst'],})
    
        df = df.replace('<BLANKFILL>',' ')
        
        columnsTitles = ['REFERENCIA','QUANT.','DATA PLAN.','DATA PROMETIDA','VALOR UNIT.',
                       'VALOR TOTAL','VALOR ICMS ST','%ICMS ST','%IPI',
                      'OPER. TRIANG.','UM','CST']

        df = df.reindex(columns=columnsTitles)
        df.index += 1
        
        result = df
        
    # Se um padrão falha, ele tenta o seguinte se houver    
    except:
        if((i+1)<=n_settings):
            extract_fields(text,settings,i+1)
            
    return result
            

In [6]:
def get_table(file):
    
    try:
        print('Usando método automático')
        table = automatic_extraction(file)
        
        return table
    
    except:
        print('Usando método manual')
        table = manual_extraction(file)
        
        return table

## Extraindo os Dados

In [7]:
get_table('exemplo1.pdf')

Usando método automático


,REFERENCIA,QUANT.,DATA PLAN.,DATA PROMETIDA,VALOR UNIT.,VALOR TOTAL,VALOR ICMS ST,%ICMS ST,%IPI,OPER. TRIANG.,UM,CST
1,BS1224,100.0,11/05/2022,18/05/2022,"5,9200","592,00",,,"5,20",N,PC,5.0
2,BS1218,200.0,11/05/2022,18/05/2022,"5,9200","1.184,00",,,"5,20",N,PC,5.0
3,KTS1-10ME-S,6.0,11/05/2022,18/05/2022,"47,4800","284,88",,,"9,75",N,CT,1.0
4,KTS34-5ME-S,6.0,11/05/2022,18/05/2022,"16,1100","96,66",,,"9,75",N,CT,1.0
5,KTS12-3ME-S,6.0,11/05/2022,18/05/2022,"8,8600","53,16",,,"9,75",N,CT,1.0
6,"MKP16X6/SK-4-2,18",5.0,11/05/2022,18/05/2022,"53,8100","269,05",,,"5,20",N,PC,0.0
7,"MKP16X6/SK-4-2,06",5.0,11/05/2022,18/05/2022,"51,4900","257,45",,,"5,20",N,PC,0.0
8,"MKP16X6/SK-4-1,77",5.0,11/05/2022,18/05/2022,"45,8200","229,10",,,"5,20",N,PC,0.0
9,"MKP16X6/SK-4-1,75",5.0,11/05/2022,03/06/2022,"45,4600","227,30",,,"5,20",N,PC,0.0
10,FCH0138-G,2.0,11/05/2022,18/05/2022,"41,7200","83,44",,,"5,20",N,PC,5.0


In [8]:
get_table('exemplo2.pdf')

Usando método automático
Usando método manual


,REFERENCIA,QUANT.,DATA PLAN.,DATA PROMETIDA,VALOR UNIT.,VALOR TOTAL,VALOR ICMS ST,%ICMS ST,%IPI,OPER. TRIANG.,UM,CST
1,"MKP16X6/SK-4-2,82",180,07/04/2022,14/04/2022,45.3100,"8,155.80",,,6.00,N,PC,0
2,"MKI16X6/SK-2,82",200,07/04/2022,14/04/2022,71.4500,"14,290.00",,,6.00,N,PC,5


In [9]:
manual_extraction('exemplo1.pdf')

,REFERENCIA,QUANT.,DATA PLAN.,DATA PROMETIDA,VALOR UNIT.,VALOR TOTAL,VALOR ICMS ST,%ICMS ST,%IPI,OPER. TRIANG.,UM,CST
1,BS1224,100,11/05/2022,18/05/2022,"5,9200","592,00",,,"5,20",N,PC,5
2,BS1218,200,11/05/2022,18/05/2022,"5,9200","1.184,00",,,"5,20",N,PC,5
3,KTS1-10ME-S,6,11/05/2022,18/05/2022,"47,4800","284,88",,,"9,75",N,CT,1
4,KTS34-5ME-S,6,11/05/2022,18/05/2022,"16,1100","96,66",,,"9,75",N,CT,1
5,KTS12-3ME-S,6,11/05/2022,18/05/2022,"8,8600","53,16",,,"9,75",N,CT,1
6,"MKP16X6/SK-4-2,18",5,11/05/2022,18/05/2022,"53,8100","269,05",,,"5,20",N,PC,0
7,"MKP16X6/SK-4-2,06",5,11/05/2022,18/05/2022,"51,4900","257,45",,,"5,20",N,PC,0
8,"MKP16X6/SK-4-1,77",5,11/05/2022,18/05/2022,"45,8200","229,10",,,"5,20",N,PC,0
9,"MKP16X6/SK-4-1,75",5,11/05/2022,03/06/2022,"45,4600","227,30",,,"5,20",N,PC,0
10,FCH0138-G,2,11/05/2022,18/05/2022,"41,7200","83,44",,,"5,20",N,PC,5
